# Visualizing The Triangle

## Read the data and build derived columns

### Import the data

In [ ]:
import pandas as pd

df=pd.read_csv('Triangle_Atlas.csv')

### Tweak it so it's more useful to us

In [ ]:
# replace NaN with string 'None' (for labels)
df.fillna(value='None', inplace=True)

df['Y']= -df['Y']  # They're all south
# So dot notation works
df.rename(columns={"Import Index": "Imports", "Export Index": "Exports"}, inplace=True) 

### Create derived columns

In [ ]:
# Symbol index:

# UFP: blue circle
# Klingon: red square
# Klingon client: darker red open square
# Romulan: green cross
# Independent: brownish open diamond
# Others: yellowish diamond

symbols={
    "Affiliation of Outer Worlds": {"color": "orange",
                                    "marker": "diamond"},
    "Imperial Klingon States": {"color": "firebrick",
                                "marker": "square-open"},
    "Independent Worlds": {"color": "darkgoldenrod",
                           "marker": "diamond-open"},
    "Klingon Empire": { "color": "red",
                       "marker": "square"},
    "Mantiev Colonial Association": {"color": "yellow",
                                     "marker": "diamond"},
    "Orion Frontier Mercantile Association": {"color": "yellowgreen",
                                              "marker": "diamond"},
    "Romulan Star Empire": {"color": "green",
                            "marker": "cross"},
    "UFP": {"color": "blue",
            "marker": "circle"}
}

ms = []
cs = []
an = []
for r in df.itertuples(index=False):
    k=r.Affiliation
    i=r.Imports
    e=r.Exports
    ms.append(symbols[k]['marker'])
    cs.append(symbols[k]['color'])
    anno = f"{r.Name}: {r.Affiliation}\n"
    anno += f"Imports: {i}\n"
    anno += f"Exports: {e}"
    an.append(anno)

### Append the columns to the right edge of the dataframe

In [ ]:
df.insert(len(df.columns), 'Marker', ms)
df.insert(len(df.columns), 'Color', cs)
df.insert(len(df.columns), 'Annotation', an)

## Plot the data

In [ ]:
import plotly.express as px

fig = px.scatter_3d(data_frame=df, x='X', y='Y', z='Z',
                    color='Color', color_discrete_map='identity',
                    symbol='Marker',
                    symbol_map='identity',
                    title='The Triangle<br><sup>Coordinate Origin: Sol System</sup>',
                    hover_data=['Name','Affiliation','Imports','Exports','X','Y','Z'],
                    width=1200, height=800
                   )

# Add the axis titles and set the aspect mode to reflect the range
fig.update_layout(scene = dict(
                  xaxis_title='pc (spinward)',
                  yaxis_title='pc (coreward)',
                  zaxis_title='pc (galactic axis)',
                  aspectmode='data'))
fig.show()
